In [1]:
import os
import pandas as pd
import sqlite3

In [2]:
conn = sqlite3.connect("airline2.db")

In [3]:
tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", conn)
print(tables)

Empty DataFrame
Columns: [name]
Index: []


In [4]:
airports = pd.read_csv("/Users/veranikapotiiko/Desktop/airports.csv")
carriers = pd.read_csv("/Users/veranikapotiiko/Desktop/carriers.csv")
planes = pd.read_csv("/Users/veranikapotiiko/Desktop/plane-data.csv")

In [5]:
file_path = "/Users/veranikapotiiko/Desktop"
csv_files = [os.path.join(file_path, f) for f in os.listdir(file_path) if f.startswith("200") and f.endswith(".csv")]
ontime = pd.concat((pd.read_csv(f) for f in csv_files), ignore_index=True)

In [6]:
airports.to_sql("airports", conn, if_exists="replace", index=False)
carriers.to_sql("carriers", conn, if_exists="replace", index=False)
planes.to_sql("planes", conn, if_exists="replace", index=False)
ontime.to_sql("ontime", conn, if_exists="replace", index=False)

24113624

In [7]:
# Quiz Q1: Airplane with the lowest average departure delay (excluding cancelled and diverted flights)

lowest_dep_delay = (
    planes.merge(ontime, left_on="tailnum", right_on="TailNum")
    .query("Cancelled == 0 & Diverted == 0 & DepDelay > 0")
    .groupby("model")["DepDelay"]
    .mean()
    .reset_index(name="avg_delay")
    .sort_values("avg_delay")
)
print(lowest_dep_delay.head(1))

      model  avg_delay
14  737-230  12.956403


In [8]:
# Quiz Q2: City with the highest number of inbound flights (excluding cancelled flights)

inbound_city = (
    airports.merge(ontime, left_on="iata", right_on="Dest")
    .query("Cancelled == 0")
    .groupby("city")
    .size()
    .reset_index(name="count")
    .sort_values("count", ascending=False)
)
print(inbound_city.head(1))

       city    count
54  Chicago  1505661


In [9]:
# Quiz Q3: Company with the highest number of cancelled flights

cancelled_flights = (
    carriers.merge(ontime, left_on="Code", right_on="UniqueCarrier")
    .query("Cancelled == 1")
    .groupby("Description")
    .size()
    .reset_index(name="total")
    .sort_values("total", ascending=False)
)
print(cancelled_flights.head(1))

                    Description  total
6  American Eagle Airlines Inc.  69529


In [10]:
# Quiz Q4: Company with the highest ratio of cancelled flights to total flights

numerator = (
    ontime.query("Cancelled == 1")
    .merge(carriers, left_on="UniqueCarrier", right_on="Code")
    .groupby("Description")
    .size()
    .reset_index(name="numerator")
)

denominator = (
    ontime.merge(carriers, left_on="UniqueCarrier", right_on="Code")
    .groupby("Description")
    .size()
    .reset_index(name="denominator")
)

ratio = numerator.merge(denominator, on="Description")
ratio["ratio"] = ratio["numerator"] / ratio["denominator"]
highest_ratio = ratio.sort_values("ratio", ascending=False).head(1)
print(highest_ratio)

                    Description  numerator  denominator     ratio
6  American Eagle Airlines Inc.      69529      1741530  0.039924


In [11]:
conn.close()